In [1]:
import os
import re
import pandas as pd

username = os.environ['JUPYTERHUB_USER']
username = re.sub(r'\.|-', '_', username) # Unsafe characters
hiveaddr = os.environ['HIVE_SERVER_2']

print("Operating as: {0}".format(username))
print("Hive server at:", hiveaddr)

Operating as: jules_gottraux
Hive server at: hadoop-2.datascience.ch


----
We can access HDFS progmmatically using the `hdfs3` python APIs. For Hive, we use `pyhive`.

In [2]:
import pandas as pd
from hdfs3 import HDFileSystem
from pyhive import hive

hdfs_host, hdfs_port = re.sub('hdfs://', '', os.environ['HADOOP_DEFAULT_FS']).split(':')
hdfs = HDFileSystem(host=hdfs_host, port=int(hdfs_port), user=username, effective_user=username)
hdfs_ls = hdfs.ls("/user/")
hdfs_ls = [re.sub('/user/', '', s) for s in hdfs_ls]
print('Users present on the Hadoop File System:', ', '.join(hdfs_ls))
print()

conn = hive.connect(host=hiveaddr, port=10000, username=username)
cursor = conn.cursor()
print('Creating table \'example\'')
cursor.execute('create database if not exists {0}'.format(username))
cursor.execute('use {0}'.format(username))
cursor.execute('drop table if exists {0}.example'.format(username))
query = """
    create external table {0}.example(
    first_name string,
    last_name string
    )
    stored as textfile
    location '/warehouse/tablespace/external/hive'
""".format(username)
cursor.execute(query)
print('Hive tables present on current database:')
pd.read_sql("show tables", conn)

Users present on the Hadoop File System: ambari-qa, hbase, hive, spark

Creating table 'example'
Hive tables present on current database:


,tab_name
0,example


#### Resources:
* Hive tutorial: [https://cwiki.apache.org/confluence/display/Hive/Tutorial](https://cwiki.apache.org/confluence/display/Hive/Tutorial)
* hdfs3 documentation: [https://hdfs3.readthedocs.io/en/latest/api.html#hdfs3.core.HDFileSystem](https://hdfs3.readthedocs.io/en/latest/api.html#hdfs3.core.HDFileSystem)
* PyHive: [https://github.com/dropbox/PyHive](https://github.com/dropbox/PyHive)